- fastapi는 무엇이고, 언제 사용을 하는지등 기본적인 이론도 같이 공부하면 좋을 것 같습니다.
  - FastAPI , uvicorn , ...

### 01. 필요한 라이브러리 설치

In [1]:
!pip install FastAPI -q
!pip install uvicorn -q
!pip install torch
!pip install python-multipart
!pip install fastapi nest-asyncio pyngrok uvicorn


[notice] A new release of pip is available: 24.1.1 -> 24.2
[notice] To update, run: python.exe -m pip install --upgrade pip

[notice] A new release of pip is available: 24.1.1 -> 24.2
[notice] To update, run: python.exe -m pip install --upgrade pip



[notice] A new release of pip is available: 24.1.1 -> 24.2
[notice] To update, run: python.exe -m pip install --upgrade pip



[notice] A new release of pip is available: 24.1.1 -> 24.2
[notice] To update, run: python.exe -m pip install --upgrade pip



[notice] A new release of pip is available: 24.1.1 -> 24.2
[notice] To update, run: python.exe -m pip install --upgrade pip


### 02. FastAPI 실행

- 학습때 설치했던 yolov5 폴더가 있는 경로로 이동해주세요.

In [12]:
import os
os.getcwd()

'A:\\commento\\2_model'

In [13]:
cd A:\commento\2_model


A:\commento\2_model


- 학습한 모델을 로드해줍니다.

In [29]:
import os
import torch
import uvicorn
import nest_asyncio

from PIL import Image
from io import BytesIO
from pyngrok import ngrok
from torchvision import transforms
from fastapi import FastAPI, File, UploadFile


# FastAPI 앱 생성
app = FastAPI()

# YOLOv5 모델 로드
model = torch.hub.load('yolov5', 'custom', path=r'A:\commento\2_model\yolov5\runs\train\yolov5s_results2\weights\best.pt', source='local')  # 학습한 모델의 경로를 'path'인자에 넣어주세요 (ex, 'yolov5/runs/train/yolov5s_results4/weights/best.pt')
model.conf = 0.5  # 신뢰도 임계값 설정

# ngrok 인증 토큰 설정
authtoken = "2mYsZHJc2N48scwLSCgcje51ehv_86wfXdYxtFkQAPrryCWy" # 자신의 인증 토큰으로 변경하세요
ngrok.set_auth_token(authtoken)

YOLOv5  v7.0-365-g12b577c8 Python-3.12.2 torch-2.3.1+cu121 CPU

Task exception was never retrieved
future: <Task finished name='Task-51' coro=<Server.serve() done, defined at c:\Users\piano\anaconda3\envs\hyojung_2015479\Lib\site-packages\uvicorn\server.py:67> exception=KeyboardInterrupt()>
Traceback (most recent call last):
  File "c:\Users\piano\anaconda3\envs\hyojung_2015479\Lib\site-packages\uvicorn\main.py", line 577, in run
    server.run()
  File "c:\Users\piano\anaconda3\envs\hyojung_2015479\Lib\site-packages\uvicorn\server.py", line 65, in run
    return asyncio.run(self.serve(sockets=sockets))
           ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "C:\Users\piano\AppData\Roaming\Python\Python312\site-packages\nest_asyncio.py", line 30, in run
    return loop.run_until_complete(task)
           ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "C:\Users\piano\AppData\Roaming\Python\Python312\site-packages\nest_asyncio.py", line 92, in run_until_complete
    self._run_once()
  File "C:\

### fastapi 실행 함수 구현
- Input : 이미지 경로
- Output : 자유롭게 아이디어를 내서 결과를 출력시켜주세요.
  - 차량 정체 문제를 파악하기 위한 값으로 출력해주세요.
  - 예를 들어, 차량 개수 파악 / 차량 전체 면적 또는 비율 등 다양한 아이디어를 생각해서 구현해주세요.

In [42]:
from PIL import ImageDraw
from fastapi.responses import FileResponse
from fastapi.responses import StreamingResponse

app = FastAPI()

# 전역 변수로 이미지 데이터를 저장할 공간
image_with_boxes_bytes = None

# 바운딩 박스를 그리는 함수
def draw_boxes(image, detections_list):
    draw = ImageDraw.Draw(image)
    for detection in detections_list:
        # 바운딩 박스 그리기
        draw.rectangle([(detection["x1"], detection["y1"]),
                        (detection["x2"], detection["y2"])], outline="red", width=3)
        # 클래스 라벨 추가
        draw.text((detection["x1"], detection["y1"] - 10), f"Class: {detection['class']}", fill="red")
    return image

# API에서 추론 후 시각화하는 코드
@app.post('/detect')
async def detect_api(file: UploadFile = File(...)):

    # 전역 변수를 업데이트할 수 있도록 global 선언
    global image_with_boxes_bytes

    # 비동기적으로 파일 읽기
    image_bytes = await file.read()

    try:
        # RGB 형태로 이미지 열기
        image = Image.open(BytesIO(image_bytes)).convert("RGB")

        # YOLOv5 모델 추론
        results = model(image)

        # 결과 처리
        detections_list = []
        df = results.pandas().xyxy[0]

        for index, row in df.iterrows():
            detections_list.append({
                "x1": float(row['xmin']),
                "y1": float(row['ymin']),
                "x2": float(row['xmax']),
                "y2": float(row['ymax']),
                "confidence": float(row['confidence']),
                "class": int(row['class'])
            })

        # 감지된 바운딩 박스(차량)의 개수
        vehicle_count = len(detections_list)

        # 바운딩 박스를 그린 이미지 반환
        image_with_boxes = draw_boxes(image, detections_list)
        
        # 메모리에서 이미지를 바이너리 형태로 저장
        image_with_boxes_bytes = BytesIO()
        image_with_boxes.save(image_with_boxes_bytes, format='JPEG')
        image_with_boxes_bytes.seek(0)

        # 이미지 파일을 /image로 요청할 수 있도록 준비
        return {
            "vehicle_count": vehicle_count,
            "detections": detections_list,
            "image_url": "/image"
        }

    except Exception as e:
        return {"error": "이미지 처리 실패", "details": str(e)}

# 이미지 반환 경로
@app.get("/image")
async def get_image():
    global image_with_boxes_bytes

    if image_with_boxes_bytes is None:
        return {"error": "이미지 파일이 존재하지 않습니다. 먼저 /detect API를 호출하세요."}
    
    # 메모리에 저장된 이미지를 반환
    image_with_boxes_bytes.seek(0)  # 이미지를 읽기 위해 처음으로 되돌림
    return StreamingResponse(image_with_boxes_bytes, media_type="image/jpeg")

# ngrok 터널 설정
port = 8000
public_url = ngrok.connect(port)
print(f"Public URL: {public_url}")

# colab에서 비동기 실행 문제 해결
nest_asyncio.apply()

# FastAPI 서버 실행
uvicorn.run(app, host="0.0.0.0", port=port)



Public URL: NgrokTunnel: "https://4f87-211-178-82-102.ngrok-free.app" -> "http://localhost:8000"


INFO:     Started server process [9136]
INFO:     Waiting for application startup.
INFO:     Application startup complete.
INFO:     Uvicorn running on http://0.0.0.0:8000 (Press CTRL+C to quit)


INFO:     211.178.82.102:0 - "GET / HTTP/1.1" 404 Not Found
INFO:     211.178.82.102:0 - "GET /favicon.ico HTTP/1.1" 404 Not Found
INFO:     211.178.82.102:0 - "GET /docs HTTP/1.1" 200 OK
INFO:     211.178.82.102:0 - "GET /openapi.json HTTP/1.1" 200 OK
INFO:     211.178.82.102:0 - "POST /detect HTTP/1.1" 200 OK
INFO:     211.178.82.102:0 - "GET /image HTTP/1.1" 200 OK


INFO:     Shutting down
INFO:     Waiting for application shutdown.
INFO:     Application shutdown complete.
INFO:     Finished server process [9136]


taskkill /F /IM ngrok.exe